# 주식 가격 예측


In [1]:
import sys
import random
import pickle

import pandas as pd
import numpy as np
import json

import math
import datetime

from sklearn import preprocessing, cross_validation, svm
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.svm import LinearSVC

from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import xgboost

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


import matplotlib.pyplot as plt


/Users/yoo/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Reading Dataset

In [2]:
# read train data
with open('./data/train_data.json') as fp:
    json_str = fp.read()
    json_data = json.loads(json_str)

In [3]:
train_df = pd.DataFrame(json_data, columns=['date','name','open','close','low','high','vol'])

In [4]:
with open('./data/grading.input.txt') as fp:
    sbm_in = fp.read()
    sbm_in = sbm_in.splitlines()

In [5]:
# teller_df = train_df[train_df['name']=='teller']
pred_lst = []
total = 0
true_t = 0
for idx, name in enumerate(sbm_in[:]):

    teller_df = train_df[train_df['name']==name][:]
    teller_df = teller_df.set_index('date')

    teller_df['hl_pct'] = (teller_df['high'] - teller_df['low'])/(teller_df['low']*100)
    teller_df['pct_chng'] = (teller_df['close'] - teller_df['open'])/(teller_df['open']*100)



    # 예측 컬럼
    forecast_col = 'close'
    # 예측 기간 하루
    forecast_out = int(1)
    # forecast_out에 맞춰 label을 변경
    teller_df['label'] = teller_df[forecast_col].shift(-forecast_out)
    # 방향 예측을 위한 전처리
    teller_df['direction'] = teller_df['label'] >= teller_df['close']

    # X, y
    X = teller_df[['close','hl_pct','pct_chng']]
    X = preprocessing.scale(X)
    X_forecast_out = X[-forecast_out:]
    X = X[:-forecast_out]

    y = np.array(teller_df['label'])
    y = y[:-forecast_out]
    y_d = np.array(teller_df['direction'])
    y_d = y_d[:-forecast_out]

    if idx % 50 == 0:
        print(idx,'counts name: ',name, '. Length of y: ', len(y))
    
    #model = SVR(kernel='linear',C=0.01).fit(X,y)
    model = LinearRegression().fit(X,y)
    model_d = LogisticRegression(C=0.1).fit(X,y_d)
    #model_d = xgboost.XGBClassifier(n_estimators=100,max_depth=7).fit(X,y_d)
    y_pred = model.predict(X_forecast_out)
    y_d_pred = model_d.predict(X_forecast_out)

    if y_d_pred:
        val = '+'
    else:
        val = '-'
   # val = '+'
    val += ' ' + str(float(y_pred))

    pred_lst.append(str(val))

0 counts name:  teller . Length of y:  249
50 counts name:  speeders . Length of y:  1760
100 counts name:  hairs . Length of y:  1760
150 counts name:  thyristors . Length of y:  1760
200 counts name:  jurisdictions . Length of y:  1760
250 counts name:  anchors . Length of y:  1760
300 counts name:  shovels . Length of y:  1760
350 counts name:  tab . Length of y:  1760
400 counts name:  tires . Length of y:  1760
450 counts name:  similarity . Length of y:  1760
500 counts name:  leaf . Length of y:  124


#### submission

In [ ]:
# submission
# 73.2714
output_file = 'submission.txt'

with open(output_file, 'w') as fp:
    for _test_pred in pred_lst:
        fp.write(_test_pred)
        fp.write('\n')